In [1]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

PATH = '../data/gold/'
STOCKS = ['AAPL', 'TSLA', 'JNJ', 'PFE', 'MARK', 'XOM', 'SO', 'NVDA', 'JPM', 'COST', 'NFLX', 'GE', 'AMT', 'T', 'AMZN']
COLS=['DATE','ID','CLOSE','VOLUME','HIGH','VOLATILITY_90D','BETA_90D','BETA_30D','BETA_9D','SMA_9D','SMA_21D','SMA_60D']

os.listdir(PATH)

['jnjbeta.csv',
 'bonds.csv',
 'tslabeta.csv',
 'pricevol_high.csv',
 'coststats.csv',
 'US10yr.csv',
 'pfebeta.csv',
 'volatility90d.csv',
 'jpmbeta.csv',
 'aaplbeta.csv',
 'gebeta.csv',
 'nvdabeta.csv']

In [2]:
df=pd.read_csv(PATH+'pricevol_high.csv')

categories = ['DATE', 'CLOSE', 'VOLUME', 'HIGH']
df.columns = pd.MultiIndex.from_product([STOCKS, categories], names=['Stock', 'Info'])
df_long = df.stack(level='Stock').reset_index()
df_long.rename(columns={'level_0': 'Original Index', 'Stock': 'ID'}, inplace=True)

pricing_df = df_long[['DATE', 'ID', 'CLOSE', 'VOLUME', 'HIGH']]
pricing_df.dropna(inplace=True)
pricing_df.DATE = pd.to_datetime(pricing_df.DATE)
pricing_df = pricing_df.loc[pricing_df.DATE > '2010-06-01']
pricing_df

Info,DATE,ID,CLOSE,VOLUME,HIGH
0,2024-05-03,AAPL,183.3800,163224109.0,187.0000
1,2024-05-03,AMT,181.7400,3130327.0,184.8900
2,2024-05-03,AMZN,186.2100,39172004.0,187.8700
3,2024-05-03,COST,743.9000,2323300.0,747.4900
4,2024-05-03,GE,164.1100,3968626.0,165.3000
...,...,...,...,...,...
52533,2010-06-02,NVDA,3.1800,64931224.0,3.1838
52534,2010-06-02,PFE,14.4088,49548599.0,14.4278
52535,2010-06-02,SO,32.6900,5779084.0,32.7100
52536,2010-06-02,T,18.7033,25630019.0,18.7109


In [3]:
df = pd.read_csv(PATH+'volatility90D.csv')
reshaped_df = pd.DataFrame()

# Extract columns in pairs and process
for i, stock in enumerate(STOCKS):
    # Calculate the column index for date and volatility
    date_col_index = 2 * i
    vol_col_index = 2 * i + 1

    # Extract data for the current stock
    stock_data = df.iloc[:, [date_col_index, vol_col_index]]
    stock_data.columns = ['DATE', 'VOLATILITY_90D']  # Standardize column names
    stock_data['ID'] = stock  # Assign stock ID

    # Append to the final DataFrame
    reshaped_df = pd.concat([reshaped_df, stock_data], ignore_index=True)

# Convert 'Date' to datetime format
reshaped_df['DATE'] = pd.to_datetime(reshaped_df['DATE'])
vola_data = reshaped_df[['DATE','ID','VOLATILITY_90D']]
vola_data.dropna(inplace=True)
vola_data = vola_data.loc[vola_data.DATE > '2010-06-01']
vola_data

,DATE,ID,VOLATILITY_90D
0,2024-05-03,AAPL,57.268
1,2024-05-02,AAPL,57.342
2,2024-05-01,AAPL,57.342
3,2024-04-30,AAPL,57.567
4,2024-04-29,AAPL,57.168
...,...,...,...
52836,2010-06-08,AMZN,35.511
52837,2010-06-07,AMZN,35.240
52838,2010-06-04,AMZN,35.519
52839,2010-06-03,AMZN,34.846


In [24]:
data = pd.merge(pricing_df, vola_data, on=['DATE', 'ID'], how='inner')
data

,DATE,ID,CLOSE,VOLUME,HIGH,VOLATILITY_90D
0,2024-05-03,AAPL,183.3800,163224109.0,187.0000,57.268
1,2024-05-03,AMT,181.7400,3130327.0,184.8900,23.369
2,2024-05-03,AMZN,186.2100,39172004.0,187.8700,26.705
3,2024-05-03,COST,743.9000,2323300.0,747.4900,20.103
4,2024-05-03,GE,164.1100,3968626.0,165.3000,29.044
...,...,...,...,...,...,...
52450,2010-06-02,NVDA,3.1800,64931224.0,3.1838,47.073
52451,2010-06-02,PFE,14.4088,49548599.0,14.4278,21.018
52452,2010-06-02,SO,32.6900,5779084.0,32.7100,15.025
52453,2010-06-02,T,18.7033,25630019.0,18.7109,16.278


In [5]:
vola_data.loc[vola_data.DATE=='2024-05-03'].loc[vola_data.ID=='GE']

,DATE,ID,VOLATILITY_90D
38764,2024-05-03,GE,29.044


In [6]:
pricing_df.loc[pricing_df.DATE=='2024-05-03'].loc[pricing_df.ID=='GE']

Info,DATE,ID,CLOSE,VOLUME,HIGH
4,2024-05-03,GE,164.11,3968626.0,165.3


In [43]:
data.VOLUME = data.VOLUME.astype(int)

In [38]:
# b = pd.read_csv(PATH+'US10yr.csv')
# b.DATE = pd.to_datetime(b.DATE)
# data = pd.merge(data, b, on='DATE', how='left')
# data.rename(columns={'YIELD': 'Risk Free Rate'}, inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52455 entries, 0 to 52454
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            52455 non-null  datetime64[ns]
 1   ID              52455 non-null  object        
 2   CLOSE           52455 non-null  float64       
 3   VOLUME          52455 non-null  int64         
 4   HIGH            52455 non-null  float64       
 5   VOLATILITY_90D  52455 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 2.4+ MB


In [33]:
df_gold = pd.read_csv(PATH+'gld.csv')
df_gold.DATE=pd.to_datetime(df_gold.DATE)
df_gold['ID'] = 'GLD'
df_gold['VOLATILITY_90D'] = pd.NA
df_gold = df_gold.loc[df_gold.DATE > '2010-06-01']

In [40]:
data = pd.concat([data,df_gold])
data.sort_values(by='DATE', ascending=False, inplace=True)
data.drop_duplicates(inplace=True)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55960 entries, 0 to 52452
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            55960 non-null  datetime64[ns]
 1   ID              55960 non-null  object        
 2   CLOSE           55960 non-null  float64       
 3   VOLUME          55960 non-null  int64         
 4   HIGH            55960 non-null  float64       
 5   VOLATILITY_90D  52455 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 3.0+ MB


In [45]:
data.to_csv('../data/bloomberg20240504.csv')